In [ ]:
%autoawait
import time
import asyncio

In [ ]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)


In [ ]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, 
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

## 1. connect to SP

In [ ]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

In [ ]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

### switch to SP 

In [ ]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

In [ ]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

In [ ]:
connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])
sp_connection = connection["connection_id"]
print(sp_connection)

In [ ]:
# sp_connection = 'e6557bae-b92b-4ad7-8c55-b91d496f6ce1'

In [ ]:
response = await agent_controller.connections.get_connections()
print(response)

In [ ]:
response = await agent_controller.connections.remove_connection('031f3c7c-1b71-49b8-aded-1fd8bea8f3d8')

## 6. connect to MUSIC

In [ ]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

### switch to MUSIC

In [ ]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

In [ ]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

In [ ]:
connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])
music_connection = connection["connection_id"]
print(music_connection)

## 7. exchange DID with SP and MUSIC

In [ ]:
my_did = await agent_controller.wallet.get_public_did()
my_did = my_did['result']['did']
print("my DID", my_did)
sp_did = 'None'
music_did = 'None'

In [ ]:
def messages_handler(payload):
    global sp_did
    global music_did
    connection_id = payload["connection_id"]
    did = payload["content"]
    if did[:3] == 'sp_':
        sp_did = did[4:]
        print('sp_did', sp_did)
    else:
        music_did = did[6:]
        print('music_did', music_did)
        
    asyncio.get_event_loop().create_task(agent_controller.messaging.send_message(connection_id, my_did))
    print("Handle message", payload, connection_id)
    
    
    

message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

In [ ]:
# message = my_did
# response = await agent_controller.messaging.send_message(music_connection, message)
# print(response)

## issue new VC to MUSIC and SP

In [ ]:
agent_controller.remove_all_listeners("basicmessages")

In [ ]:
revoke = False

In [ ]:
def request_handler(payload):
    global revoke
    connection_id = payload["connection_id"]
    if payload['content'] == 'True': 
        revoke = True
    send_task = asyncio.get_event_loop().create_task(agent_controller.messaging.send_message(sp_connection, "revoke: "+str(revoke)))
    print("Handle request", payload, connection_id)


request_listener = {
    "handler": request_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([request_listener], defaults=True)

### switch to SP

In [ ]:
print(revoke)

### revoke old VC from MUSIC

In [ ]:
if revoke:
    ## revoke the previous VC
    

### issue new VC to MUSIC

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

In [ ]:
created_schema = await agent_controller.schema.get_created_schema(schema_issuer_did=my_did)
print(created_schema)
schema_id = created_schema['schema_ids'][0]
print(schema_id)

## find schema_id

In [ ]:
created_cred_def = response = await agent_controller.definitions.search_created(schema_issuer_did=my_did)
print(created_cred_def)
cred_def_id = created_cred_def['credential_definition_ids'][0]
print(cred_def_id)

## find cred_def_id

In [ ]:
import datetime
time = datetime.datetime.now().strftime("%m-%d-%Y, %H:%M:%S")
credential_attributes = [
    {"name": "author", "value": my_did},
    {"name": "origin", "value": my_did},
    {"name": "owner", "value": sp_did},
    {"name": "time", "value": time},
    {"name": "version", "value": "v1"},
    {"name": "name", "value": "balabababa music"},
    ]
record = await agent_controller.issuer.send_credential(music_connection, schema_id, cred_def_id, credential_attributes, auto_remove=False, trace=True)

In [ ]:
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

### define new schema 

In [ ]:
# Define you schema name - must be unique on the ledger
schema_name = "possession schema"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["possession", "time"]


In [ ]:
response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

In [ ]:
response = await agent_controller.definitions.write_cred_def(schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)

### issue new VC to SP

In [ ]:
import datetime
time = datetime.datetime.now().strftime("%m-%d-%Y, %H:%M:%S")
credential_attributes = [
    {"name": "possession", "value": music_did},
    {"name": "time", "value": time},
]
print(credential_attributes)

In [ ]:
record = await agent_controller.issuer.send_credential(sp_connection, schema_id, cred_def_id, credential_attributes, auto_remove=False, trace=True)
print(record)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

In [ ]:
response = await agent_controller.terminate()
print(response)